In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/train.csv')
test = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/test.csv')
sub = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')

In [ ]:
train = train.sample(1000)
train

In [ ]:
train2 = train.drop(columns = ['id', 'target'])
test2 = test.drop(columns ='id')

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
xgb  = XGBClassifier(learning_rate = 0.1)

# lgb = LGBMClassifier(num_leaves = 31, n_estimators = 200, learning_rate = 0.05)
lgb = LGBMClassifier(max_depth = 7, n_estimators = 500, learning_rate = 0.1)
cbc = CatBoostClassifier(verbose = 100)
rf = RandomForestClassifier()

In [ ]:
xgb.fit(train2, train['target'])
# lgb.fit(train2, train['target'])
cbc.fit(train2, train['target'])
rf.fit(train2, train['target'])

In [ ]:
result_xgb = xgb.predict_proba(test2)
# result_lgb = lgb.predict_proba(test2)
result_cbc = cbc.predict_proba(test2)
result_rf = rf.predict_proba(test2)

In [ ]:
sub.iloc[:,1:] = result_xgb
sub.to_csv('xgb.csv', index = 0)

In [ ]:
sub2 = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')
sub3 = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')
sub4 = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')

In [ ]:
sub2.iloc[:,1:] = result_rf
sub2.to_csv('rf.csv', index = 0)

In [ ]:
sub3.iloc[:,1:] = result_cbc
sub3.to_csv('cbc.csv', index = 0)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# 앙상블 - voting, blending ,stacking
# final_estimator --> 예측값을 final_estimator 칼럼으로 보냄
# lv1 에서 나온 예측값들로 lv2에서 예측값을 새로운 input 으로 해서 새로 돌리는것. meta model, meta learner 임.

sc = StackingClassifier(n_jobs = -1, estimators = [('xgb',xgb),('rf',rf),('cbc',cbc)], final_estimator = LogisticRegression(), verbose = 1)

sc.fit(train2, train['target'])

In [ ]:
result_sc = sc.predict_proba(test2)

In [ ]:
sub4.iloc[:,1:] = result_sc
sub4.to_csv('stacking.csv', index = 0)